In [16]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession

In [18]:
spark = SparkSession\
    .builder\
    .master("local")\
    .appName("word_count_jsonl")\
    .config("spark.some.config.option", "some-value")\
    .getOrCreate()

In [2]:
path = '~/mini_pyspark_projects/article_samples.json.gz'